# Deploy trained and registered model

### Connect to workspace

In [110]:
# Connect to workspace
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, "Workspace loaded")

zpdsnfws02 Workspace loaded


### Select model to deploy in workspace

In [111]:
# Get models registered in workspace
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, "version:", model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print("\t", tag_name, ":", tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print("\t", prop_name, ":", prop)
    print("\n")

marketing_request_20210312_model version: 5
	 Training context : Inline Training
	 MSE : 134839444.443378
	 Accuracy : 0.0


marketing_request_20210312_model_v1 version: 1
	 Training context : Inline Training
	 MSE : 134839444.443378
	 Accuracy : 0.0


marketing_request_20210312_model version: 4
	 Training context : Inline Training
	 MSE : 134839444.443378
	 Accuracy : 0.0


marketing_request_20210312_model version: 3
	 Training context : Inline Training
	 MSE : 134839444.443378
	 Accuracy : 0.0


marketing_request_20210312_model version: 2
	 Training context : Inline Training
	 MSE : 134839444.443378
	 Accuracy : 0.0


marketing_request_20210312_model version: 1
	 Training context : Inline Training
	 MSE : 134839444.443378
	 Accuracy : 0.0


ApplicationCategoryRunHistory.mml version: 10


ApplicationCategoryRunHistory.mml version: 9


ApplicationCategoryRunHistory.mml version: 8


ApplicationCategoryRunHistory.mml version: 7


ApplicationCategoryRunHistory.mml version: 6


Application

In [112]:
# Select model to deploy
model = ws.models["marketing_request_20210312_model"]
print(model.name, "version", model.version)

marketing_request_20210312_model version 5


In [113]:
model.id

'marketing_request_20210312_model:5'

### Create folder for webservice

#### Create folder to store files for webservice

In [114]:
# Create folder to store code and config files to create webservice
import os

folder_name = "marketing_request_20210312_service"
experiment_folder = "./" + folder_name
os.makedirs(experiment_folder, exist_ok = True)
print(experiment_folder, "folder created")

./marketing_request_20210312_service folder created


#### Write scoring script

In [115]:
%%writefile $experiment_folder/score_marketing_request_20210312.py
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global myModel
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path(model_name = 'marketing_request_20210312_model')
    myModel = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)["data"])
    # Get a prediction from the model
    predictions = myModel.predict(data)
    prediction_list = []
    for prediction in predictions:
        prediction_list.append(prediction)
    # Return the predictions as JSON
    return json.dumps(prediction_list)


Overwriting ./marketing_request_20210312_service/score_marketing_request_20210312.py


#### Create YAML file for dependencies

In [125]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_pip_package("scikit-learn==0.21.3")
myenv.add_pip_package("pandas")
myenv.add_pip_package("pyspark")

# Save the environment config as a .yml file
env_file = experiment_folder + "/marketing_request_20210312_env.yml"
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

Saved dependency info in ./marketing_request_20210312_service/marketing_request_20210312_env.yml
# Conda environment specification. The dependencies defined in this file will

# be automatically provisioned for runs with userManagedDependencies=False.


# Details about the Conda environment file format:

# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually


name: project_environment
dependencies:
  # The python interpreter version.

  # Currently Azure ML only supports 3.5.2 and later.

- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.

  - azureml-defaults

  - scikit-learn==0.21.3
  - pandas
  - pyspark
channels:
- anaconda
- conda-forge



### Deploy model

In [126]:
# Deploy
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   source_directory = experiment_folder,
                                   entry_script="score_marketing_request_20210312.py",
                                   conda_file="marketing_request_20210312_env.yml")

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "marketing-request-service10"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-09-30 17:32:40+08:00 Creating Container Registry if not exists.
2021-09-30 17:32:41+08:00 Registering the environment.
2021-09-30 17:32:43+08:00 Building image..
2021-09-30 17:37:56+08:00 Generating deployment configuration.
2021-09-30 17:37:57+08:00 Submitting deployment to compute..
2021-09-30 17:38:04+08:00 Checking the status of deployment marketing-request-service10..
2021-09-30 17:40:17+08:00 Checking the status of inference endpoint marketing-request-service10.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [127]:
print(service.state)
print(service.get_logs())

# If you need to make a change and redeploy, you may need to delete unhealthy service using the following code:
#service.delete()

Healthy
2021-09-30T09:40:06,832743800+00:00 - rsyslog/run 
2021-09-30T09:40:06,831726300+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-09-30T09:40:06,838219100+00:00 - iot-server/run 
2021-09-30T09:40:06,958889700+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-09-30T09:40:07,410933100+00:00 - iot-server/finish 1 0
2021-09-30T09:40:07,417435200+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (69)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 97
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-09-30 09:40:10,254 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2021-09-30 09:40:10,255 | root | INFO | Starting up request id generator
2021-09-30 09:40:10,255 | root | INF

In [128]:
for webservice_name in ws.webservices:
    print(webservice_name)

marketing-request-service10
marketing-request-service9
marketing-request-service8
marketing-request-service7
marketing-request-service6
marketing-request-service5
marketing-request-service4
marketing-request-service3
marketing-request-service2
marketing-request-service
applicationcategorywebservice


#### Get endpoint (URI)

In [129]:
endpoint = service.scoring_uri
print(endpoint)

http://d27dd023-0572-413d-b8a2-1b6e06b75de4.southeastasia.azurecontainer.io/score


#### Get predictions through REST API i.e. endpoint

In [131]:
import requests
import json

x_new = [[2,180],
         [22,148]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_scores = json.loads(predictions.json())
print(predictions)
print(predicted_scores)

<Response [200]>
[122.8123770091251, 122.86054712915386]
